In [4]:
import math
import numpy as np
import pandas as pd
from collections import Counter

In [5]:
np.random.seed(seed=408)

In [3]:
# 1) Get elo functions to work
# 2) Get elo function to take from dataframe
# 3) Get elo function to write to dataframe
# 4) Add in all remaining regular season games to simulate as a function
# 5) Get final regular season dataframe

# 6) Create final standings df for each conference
# 7) Create new function to simulate play-in games if applicable (four games back in either conference)

# 8) Get final post-season bracket and create postseason dataframe
# 9) Simulate each game while updating elo
# 10) Make bracket move on to next step
# 11) Decide NBA champion
# 12) Run loop to track number of times each team has won in 10000 simulations

In [6]:
# Create df with team name, wins, losses, elo
# team_df = {'Team': ['Lakers', 'Clippers'],
#           'Wins': [50, 45],
#           'Losses': [10, 15],
#           'Elo': [1800, 1750]}
team_df = pd.read_csv("C:\\Users\\Divya Parmar's PC\\Documents\\NBA-Playoffs-2020-Simulation-V1\\nba_elo_data.csv")
team_df['elo_value'] = 0.5 * team_df['elo_basic'] + 0.5 * team_df['elo_playoff_full']
team_df_testing = team_df
team_df_backup = team_df

In [7]:
# Look at our dataframe
team_df

,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value
0,LAL,Lakers,1678,1710,49,14,western,1694.0
1,LAC,Clippers,1636,1732,44,20,western,1684.0
2,MIL,Bucks,1716,1694,53,12,eastern,1705.0
3,PHI,76ers,1564,1691,39,26,eastern,1627.5
4,HOU,Rockets,1565,1701,40,24,western,1633.0
5,BOS,Celtics,1628,1682,43,21,eastern,1655.0
6,TOR,Raptors,1679,1628,46,18,eastern,1653.5
7,DEN,Nuggets,1579,1638,43,22,western,1608.5
8,OKC,Thunder,1608,1583,40,24,western,1595.5
9,DAL,Mavericks,1584,1643,40,27,western,1613.5


In [8]:
# First let's write a function to calculate elo
# Thankfully it's  already been done
# https://www.geeksforgeeks.org/elo-rating-algorithm/

In [9]:
def elo_probability(rating1, rating2):
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating2 - rating1)/400))

In [10]:
# Validate that the probabilities add up to 1 for a given matchup
elo_probability(2000, 1200) + elo_probability(1200, 2000)

1.0

In [11]:
elo_probability(1200, 2000)

0.009900990099009901

In [9]:
# K-Factor determines how quickly elo score reacts to new events
# FiveThirtyEight recommends k-factor of 20 for NBA games
k = 20

In [13]:
def game_winner(rating1, rating2):
    prob = elo_probability(rating1, rating2)
    random_value = np.random.uniform(0,1)
    if random_value <= prob:
        return 1
    else:
        return 2

In [17]:
# Unit test of game_winner function
game_winner(1800, 1700)

1

In [19]:
def elo_update(rating_a, rating_b):
    Pa = elo_probability(rating_a, rating_b)
    Pb = elo_probability(rating_b, rating_a)
    
    game_winner_output = game_winner(rating_a, rating_b)
    
    if game_winner_output == 1:
        rating_a_updated = rating_a + 20 * (1-Pa)
        rating_b_updated = rating_b + 20 * (0-Pb)
    
    elif game_winner_output == 2:
        rating_a_updated = rating_a + 20 * (0-Pa)
        rating_b_updated = rating_b + 20 * (1-Pb)
    
    else:
        rating_a_updated = rating_a
        rating_b_updated = rating_b
    return rating_a_updated, rating_b_updated, game_winner_output

In [20]:
# Unit test elo_update
test_elo_a, test_elo_b, winner = elo_update(1800, 1700)
print(test_elo_a, test_elo_b, winner)

(1807.1987000039423, 1692.8012999960577, 1)


In [14]:
def game_simulation_regular_season(team_a, team_b, input_df):
    #Get elo values from database
    elo_a = input_df['elo_value'][input_df['team_abbrev'] == team_a].values[0]
    elo_b = input_df['elo_value'][input_df['team_abbrev'] == team_b].values[0]
    #Run elo update and game winner functions
    new_elo_a, new_elo_b, game_winner_output = elo_update(elo_a, elo_b)
    #Update elo for first team
    input_df['elo_value'][input_df['team_abbrev'] == team_a] = new_elo_a
    #Update elo for second team
    input_df['elo_value'][input_df['team_abbrev'] == team_b] = new_elo_b
    #Add wins and losses to respective teams
    if game_winner_output == 1:
        input_df['wins'][input_df['team_abbrev'] == team_a] += 1
        input_df['losses'][input_df['team_abbrev'] == team_b] += 1
    elif game_winner_output == 2:
        input_df['wins'][input_df['team_abbrev'] == team_b] += 1
        input_df['losses'][input_df['team_abbrev'] == team_a] += 1
    else:
        pass
    return input_df

In [15]:
# Unit test game_simulation function
game_simulation_regular_season('LAL', 'LAC', team_df_testing)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.p

,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value
0,LAL,Lakers,1678,1710,49,15,western,1683.712256
1,LAC,Clippers,1636,1732,45,20,western,1694.287744
2,MIL,Bucks,1716,1694,53,12,eastern,1705.000000
3,PHI,76ers,1564,1691,39,26,eastern,1627.500000
4,HOU,Rockets,1565,1701,40,24,western,1633.000000
5,BOS,Celtics,1628,1682,43,21,eastern,1655.000000
6,TOR,Raptors,1679,1628,46,18,eastern,1653.500000
7,DEN,Nuggets,1579,1638,43,22,western,1608.500000
8,OKC,Thunder,1608,1583,40,24,western,1595.500000
9,DAL,Mavericks,1584,1643,40,27,western,1613.500000


In [16]:
# Simulate all the regular season games
# Get projected game schedule here:
# https://nba.nbcsports.com/2020/06/03/nba-playoff-schedule-projecting-for-all-22-returning-nba-teams/
# UPDATE: We have an actual schedule
# https://twitter.com/ChrisBHaynes/status/1276655999420526592/photo/1

In [17]:
#Day1
team_df = game_simulation_regular_season('UTA', 'NOP', team_df)
team_df = game_simulation_regular_season('LAC', 'LAL', team_df)
team_df = game_simulation_regular_season('ORL', 'BKN', team_df)
#Day2
team_df = game_simulation_regular_season('ORL', 'BKN', team_df)
team_df = game_simulation_regular_season('MEM', 'POR', team_df)
team_df = game_simulation_regular_season('PHX', 'WAS', team_df)
team_df = game_simulation_regular_season('BOS', 'MIL', team_df)
team_df = game_simulation_regular_season('SAC', 'SAS', team_df)
team_df = game_simulation_regular_season('HOU', 'DAL', team_df)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [18]:
#Day3
team_df = game_simulation_regular_season('MIA', 'DEN', team_df)
team_df = game_simulation_regular_season('UTA', 'OKC', team_df)
team_df = game_simulation_regular_season('NOP', 'LAC', team_df)
team_df = game_simulation_regular_season('PHI', 'IND', team_df)
team_df = game_simulation_regular_season('LAL', 'TOR', team_df)
#Day4
team_df = game_simulation_regular_season('WAS', 'BKN', team_df)
team_df = game_simulation_regular_season('POR', 'BOS', team_df)
team_df = game_simulation_regular_season('SAS', 'MEM', team_df)
team_df = game_simulation_regular_season('SAC', 'ORL', team_df)
team_df = game_simulation_regular_season('MIL', 'HOU', team_df)
team_df = game_simulation_regular_season('DAL', 'PHX', team_df)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [19]:
#Day5
team_df = game_simulation_regular_season('TOR', 'MIA', team_df)
team_df = game_simulation_regular_season('DEN', 'OKC', team_df)
team_df = game_simulation_regular_season('IND', 'WAS', team_df)
team_df = game_simulation_regular_season('MEM', 'NOP', team_df)
team_df = game_simulation_regular_season('SAS', 'PHI', team_df)
team_df = game_simulation_regular_season('LAL', 'UTA', team_df)
#Day6
team_df = game_simulation_regular_season('BKN', 'MIL', team_df)
team_df = game_simulation_regular_season('DAL', 'SAC', team_df)
team_df = game_simulation_regular_season('PHX', 'LAC', team_df)
team_df = game_simulation_regular_season('ORL', 'IND', team_df)
team_df = game_simulation_regular_season('BOS', 'MIA', team_df)
team_df = game_simulation_regular_season('HOU', 'POR', team_df)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [20]:
#Day7
team_df = game_simulation_regular_season('MEM', 'UTA', team_df)
team_df = game_simulation_regular_season('PHI', 'WAS', team_df)
team_df = game_simulation_regular_season('DEN', 'SAS', team_df)
team_df = game_simulation_regular_season('OKC', 'LAL', team_df)
team_df = game_simulation_regular_season('TOR', 'ORL', team_df)
team_df = game_simulation_regular_season('BKN', 'BOS', team_df)
#Day8
team_df = game_simulation_regular_season('NOP', 'SAC', team_df)
team_df = game_simulation_regular_season('MIA', 'MIL', team_df)
team_df = game_simulation_regular_season('IND', 'PHX', team_df)
team_df = game_simulation_regular_season('LAC', 'DAL', team_df)
team_df = game_simulation_regular_season('POR', 'DEN', team_df)
team_df = game_simulation_regular_season('LAL', 'HOU', team_df)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [21]:
#Day9
team_df = game_simulation_regular_season('UTA', 'SAS', team_df)
team_df = game_simulation_regular_season('OKC', 'MEM', team_df)
team_df = game_simulation_regular_season('SAC', 'BKN', team_df)
team_df = game_simulation_regular_season('ORL', 'PHI', team_df)
team_df = game_simulation_regular_season('WAS', 'NOP', team_df)
team_df = game_simulation_regular_season('BOS', 'TOR', team_df)
#Day10
team_df = game_simulation_regular_season('LAC', 'POR', team_df)
team_df = game_simulation_regular_season('UTA', 'DEN', team_df)
team_df = game_simulation_regular_season('LAL', 'IND', team_df)
team_df = game_simulation_regular_season('PHX', 'MIA', team_df)
team_df = game_simulation_regular_season('MIL', 'DAL', team_df)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [22]:
#Day11
team_df = game_simulation_regular_season('WAS', 'OKC', team_df)
team_df = game_simulation_regular_season('MEM', 'TOR', team_df)
team_df = game_simulation_regular_season('SAS', 'NOP', team_df)
team_df = game_simulation_regular_season('ORL', 'BOS', team_df)
team_df = game_simulation_regular_season('PHI', 'POR', team_df)
team_df = game_simulation_regular_season('HOU', 'SAC', team_df)
team_df = game_simulation_regular_season('BKN', 'LAC', team_df)
#Day12
team_df = game_simulation_regular_season('OKC', 'PHX', team_df)
team_df = game_simulation_regular_season('DAL', 'UTA', team_df)
team_df = game_simulation_regular_season('TOR', 'MIL', team_df)
team_df = game_simulation_regular_season('IND', 'MIA', team_df)
team_df = game_simulation_regular_season('DEN', 'LAL', team_df)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [23]:
#Day13
team_df = game_simulation_regular_season('BKN', 'ORL', team_df)
team_df = game_simulation_regular_season('HOU', 'SAS', team_df)
team_df = game_simulation_regular_season('PHX', 'PHI', team_df)
team_df = game_simulation_regular_season('POR', 'DAL', team_df)
team_df = game_simulation_regular_season('BOS', 'MEM', team_df)
team_df = game_simulation_regular_season('NOP', 'SAC', team_df)
team_df = game_simulation_regular_season('MIL', 'WAS', team_df)
#Day14
team_df = game_simulation_regular_season('IND', 'HOU', team_df)
team_df = game_simulation_regular_season('TOR', 'PHI', team_df)
team_df = game_simulation_regular_season('MIA', 'OKC', team_df)
team_df = game_simulation_regular_season('LAC', 'DEN', team_df)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [24]:
#Day15
team_df = game_simulation_regular_season('WAS', 'BOS', team_df)
team_df = game_simulation_regular_season('POR', 'BKN', team_df)
team_df = game_simulation_regular_season('SAC', 'LAL', team_df)
team_df = game_simulation_regular_season('MIL', 'MEM', team_df)
team_df = game_simulation_regular_season('NOP', 'ORL', team_df)
team_df = game_simulation_regular_season('DAL', 'PHX', team_df)
team_df = game_simulation_regular_season('SAS', 'UTA', team_df)
#Day16
team_df = game_simulation_regular_season('PHI', 'HOU', team_df)
team_df = game_simulation_regular_season('MIA', 'IND', team_df)
team_df = game_simulation_regular_season('OKC', 'LAC', team_df)
team_df = game_simulation_regular_season('DEN', 'TOR', team_df)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [49]:
def all_regular_season(team_df):
    #Day1
    team_df = game_simulation_regular_season('UTA', 'NOP', team_df)
    team_df = game_simulation_regular_season('LAC', 'LAL', team_df)
    team_df = game_simulation_regular_season('ORL', 'BKN', team_df)
    #Day2
    team_df = game_simulation_regular_season('ORL', 'BKN', team_df)
    team_df = game_simulation_regular_season('MEM', 'POR', team_df)
    team_df = game_simulation_regular_season('PHX', 'WAS', team_df)
    team_df = game_simulation_regular_season('BOS', 'MIL', team_df)
    team_df = game_simulation_regular_season('SAC', 'SAS', team_df)
    team_df = game_simulation_regular_season('HOU', 'DAL', team_df)
    #Day3
    team_df = game_simulation_regular_season('MIA', 'DEN', team_df)
    team_df = game_simulation_regular_season('UTA', 'OKC', team_df)
    team_df = game_simulation_regular_season('NOP', 'LAC', team_df)
    team_df = game_simulation_regular_season('PHI', 'IND', team_df)
    team_df = game_simulation_regular_season('LAL', 'TOR', team_df)
    #Day4
    team_df = game_simulation_regular_season('WAS', 'BKN', team_df)
    team_df = game_simulation_regular_season('POR', 'BOS', team_df)
    team_df = game_simulation_regular_season('SAS', 'MEM', team_df)
    team_df = game_simulation_regular_season('SAC', 'ORL', team_df)
    team_df = game_simulation_regular_season('MIL', 'HOU', team_df)
    team_df = game_simulation_regular_season('DAL', 'PHX', team_df)
    #Day5
    team_df = game_simulation_regular_season('TOR', 'MIA', team_df)
    team_df = game_simulation_regular_season('DEN', 'OKC', team_df)
    team_df = game_simulation_regular_season('IND', 'WAS', team_df)
    team_df = game_simulation_regular_season('MEM', 'NOP', team_df)
    team_df = game_simulation_regular_season('SAS', 'PHI', team_df)
    team_df = game_simulation_regular_season('LAL', 'UTA', team_df)
    #Day6
    team_df = game_simulation_regular_season('BKN', 'MIL', team_df)
    team_df = game_simulation_regular_season('DAL', 'SAC', team_df)
    team_df = game_simulation_regular_season('PHX', 'LAC', team_df)
    team_df = game_simulation_regular_season('ORL', 'IND', team_df)
    team_df = game_simulation_regular_season('BOS', 'MIA', team_df)
    team_df = game_simulation_regular_season('HOU', 'POR', team_df)
    #Day7
    team_df = game_simulation_regular_season('MEM', 'UTA', team_df)
    team_df = game_simulation_regular_season('PHI', 'WAS', team_df)
    team_df = game_simulation_regular_season('DEN', 'SAS', team_df)
    team_df = game_simulation_regular_season('OKC', 'LAL', team_df)
    team_df = game_simulation_regular_season('TOR', 'ORL', team_df)
    team_df = game_simulation_regular_season('BKN', 'BOS', team_df)
    #Day8
    team_df = game_simulation_regular_season('NOP', 'SAC', team_df)
    team_df = game_simulation_regular_season('MIA', 'MIL', team_df)
    team_df = game_simulation_regular_season('IND', 'PHX', team_df)
    team_df = game_simulation_regular_season('LAC', 'DAL', team_df)
    team_df = game_simulation_regular_season('POR', 'DEN', team_df)
    team_df = game_simulation_regular_season('LAL', 'HOU', team_df)
    #Day9
    team_df = game_simulation_regular_season('UTA', 'SAS', team_df)
    team_df = game_simulation_regular_season('OKC', 'MEM', team_df)
    team_df = game_simulation_regular_season('SAC', 'BKN', team_df)
    team_df = game_simulation_regular_season('ORL', 'PHI', team_df)
    team_df = game_simulation_regular_season('WAS', 'NOP', team_df)
    team_df = game_simulation_regular_season('BOS', 'TOR', team_df)
    #Day10
    team_df = game_simulation_regular_season('LAC', 'POR', team_df)
    team_df = game_simulation_regular_season('UTA', 'DEN', team_df)
    team_df = game_simulation_regular_season('LAL', 'IND', team_df)
    team_df = game_simulation_regular_season('PHX', 'MIA', team_df)
    team_df = game_simulation_regular_season('MIL', 'DAL', team_df)
    #Day11
    team_df = game_simulation_regular_season('WAS', 'OKC', team_df)
    team_df = game_simulation_regular_season('MEM', 'TOR', team_df)
    team_df = game_simulation_regular_season('SAS', 'NOP', team_df)
    team_df = game_simulation_regular_season('ORL', 'BOS', team_df)
    team_df = game_simulation_regular_season('PHI', 'POR', team_df)
    team_df = game_simulation_regular_season('HOU', 'SAC', team_df)
    team_df = game_simulation_regular_season('BKN', 'LAC', team_df)
    #Day12
    team_df = game_simulation_regular_season('OKC', 'PHX', team_df)
    team_df = game_simulation_regular_season('DAL', 'UTA', team_df)
    team_df = game_simulation_regular_season('TOR', 'MIL', team_df)
    team_df = game_simulation_regular_season('IND', 'MIA', team_df)
    team_df = game_simulation_regular_season('DEN', 'LAL', team_df)
    #Day13
    team_df = game_simulation_regular_season('BKN', 'ORL', team_df)
    team_df = game_simulation_regular_season('HOU', 'SAS', team_df)
    team_df = game_simulation_regular_season('PHX', 'PHI', team_df)
    team_df = game_simulation_regular_season('POR', 'DAL', team_df)
    team_df = game_simulation_regular_season('BOS', 'MEM', team_df)
    team_df = game_simulation_regular_season('NOP', 'SAC', team_df)
    team_df = game_simulation_regular_season('MIL', 'WAS', team_df)
    #Day14
    team_df = game_simulation_regular_season('IND', 'HOU', team_df)
    team_df = game_simulation_regular_season('TOR', 'PHI', team_df)
    team_df = game_simulation_regular_season('MIA', 'OKC', team_df)
    team_df = game_simulation_regular_season('LAC', 'DEN', team_df)
    #Day15
    team_df = game_simulation_regular_season('WAS', 'BOS', team_df)
    team_df = game_simulation_regular_season('POR', 'BKN', team_df)
    team_df = game_simulation_regular_season('SAC', 'LAL', team_df)
    team_df = game_simulation_regular_season('MIL', 'MEM', team_df)
    team_df = game_simulation_regular_season('NOP', 'ORL', team_df)
    team_df = game_simulation_regular_season('DAL', 'PHX', team_df)
    team_df = game_simulation_regular_season('SAS', 'UTA', team_df)
    #Day16
    team_df = game_simulation_regular_season('PHI', 'HOU', team_df)
    team_df = game_simulation_regular_season('MIA', 'IND', team_df)
    team_df = game_simulation_regular_season('OKC', 'LAC', team_df)
    team_df = game_simulation_regular_season('DEN', 'TOR', team_df)
    #Output
    return team_df

In [26]:
team_df

,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value
0,LAL,Lakers,1678,1710,56,16,western,1724.181604
1,LAC,Clippers,1636,1732,51,22,western,1703.862966
2,MIL,Bucks,1716,1694,58,15,eastern,1693.291963
3,PHI,76ers,1564,1691,45,28,eastern,1650.901135
4,HOU,Rockets,1565,1701,43,29,western,1604.060336
5,BOS,Celtics,1628,1682,46,26,eastern,1613.110102
6,TOR,Raptors,1679,1628,51,21,eastern,1662.524108
7,DEN,Nuggets,1579,1638,46,27,western,1586.534914
8,OKC,Thunder,1608,1583,43,29,western,1574.761197
9,DAL,Mavericks,1584,1643,42,33,western,1575.261187


In [27]:
# Create win percentage column
team_df['win_percentage'] = team_df['wins']/(team_df['wins']+team_df['losses'])

In [28]:
# Create conference specific dataframes
df_western = team_df[team_df['conference'] == 'western']
df_eastern = team_df[team_df['conference'] == 'eastern']

In [29]:
# Get Western standings at the end of the regular season
df_western['standing_min'] = df_western['win_percentage'].rank(method='min', ascending=False)
df_western['standing_dense'] = df_western['win_percentage'].rank(method='dense', ascending=False)
df_western['standing_random'] = df_western['win_percentage'].rank(method='first', ascending=False)
df_western.sort_values(by=['standing_min', 'standing_random'], inplace=True)
df_western.reset_index(inplace=True)
df_western['ahead_wins'] = df_western['wins'].shift(1)
df_western['ahead_losses'] = df_western['losses'].shift(1)

df_western['games_back_of_ahead_team'] = ((df_western['ahead_wins'] - df_western['wins']) + (df_western['losses'] - df_western['ahead_losses']))/2

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [30]:
df_western

,index,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,win_percentage,standing_min,standing_dense,standing_random,ahead_wins,ahead_losses,games_back_of_ahead_team
0,0,LAL,Lakers,1678,1710,56,16,western,1724.181604,0.777778,1.0,1.0,1.0,NaN,NaN,NaN
1,1,LAC,Clippers,1636,1732,51,22,western,1703.862966,0.698630,2.0,2.0,2.0,56.0,16.0,5.5
2,11,UTA,Jazz,1574,1584,48,24,western,1631.234866,0.666667,3.0,3.0,3.0,51.0,22.0,2.5
3,7,DEN,Nuggets,1579,1638,46,27,western,1586.534914,0.630137,4.0,4.0,4.0,48.0,24.0,2.5
4,4,HOU,Rockets,1565,1701,43,29,western,1604.060336,0.597222,5.0,5.0,5.0,46.0,27.0,2.5
5,8,OKC,Thunder,1608,1583,43,29,western,1574.761197,0.597222,5.0,5.0,6.0,43.0,29.0,0.0
6,9,DAL,Mavericks,1584,1643,42,33,western,1575.261187,0.560000,7.0,6.0,7.0,43.0,29.0,2.5
7,17,MEM,Grizzlies,1553,1456,35,38,western,1506.238907,0.479452,8.0,7.0,8.0,42.0,33.0,6.0
8,10,NOP,Pelicans,1542,1611,33,39,western,1585.315911,0.458333,9.0,8.0,9.0,35.0,38.0,1.5
9,14,POR,Trailblazers,1490,1569,32,42,western,1523.966998,0.432432,10.0,9.0,10.0,33.0,39.0,2.0


In [31]:
# Get Eastern standings at the end of the regular season
df_eastern['standing_min'] = df_eastern['win_percentage'].rank(method='min', ascending=False)
df_eastern['standing_dense'] = df_eastern['win_percentage'].rank(method='dense', ascending=False)
df_eastern['standing_random'] = df_eastern['win_percentage'].rank(method='first', ascending=False)
df_eastern.sort_values(by=['standing_min', 'standing_random'], inplace=True)
df_eastern.reset_index(inplace=True)
df_eastern['ahead_wins'] = df_eastern['wins'].shift(1)
df_eastern['ahead_losses'] = df_eastern['losses'].shift(1)

df_eastern['games_back_of_ahead_team'] = ((df_eastern['ahead_wins'] - df_eastern['wins']) + (df_eastern['losses'] - df_eastern['ahead_losses']))/2

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [32]:
def bracket_cleaner(input_df):
    input_df['standing_min'] = input_df['win_percentage'].rank(method='min', ascending=False)
    input_df['standing_dense'] = input_df['win_percentage'].rank(method='dense', ascending=False)
    input_df['standing_random'] = input_df['win_percentage'].rank(method='first', ascending=False)
    input_df.sort_values(by=['standing_min', 'standing_random'], inplace=True)
    input_df.reset_index(inplace=True)
    input_df['ahead_wins'] = input_df['wins'].shift(1)
    input_df['ahead_losses'] = input_df['losses'].shift(1)
    input_df['games_back_of_ahead_team'] = ((input_df['ahead_wins'] - input_df['wins']) + (input_df['losses'] - input_df['ahead_losses']))/2
    return input_df

In [33]:
# Create series for each Western seed 1-7 and Eastern seed 1-7
west_seed_one = df_western.loc[0]
west_seed_two = df_western.loc[1]
west_seed_three = df_western.loc[2]
west_seed_four = df_western.loc[3]
west_seed_five = df_western.loc[4]
west_seed_six = df_western.loc[5]
west_seed_seven = df_western.loc[6]
east_seed_one = df_eastern.loc[0]
east_seed_two = df_eastern.loc[1]
east_seed_three = df_eastern.loc[2]
east_seed_four = df_eastern.loc[3]
east_seed_five = df_eastern.loc[4]
east_seed_six = df_eastern.loc[5]
east_seed_seven = df_eastern.loc[6]

In [34]:
# Play-in round function
def play_in_round(input_df):
    eight = input_df.loc[7]
    nine = input_df.loc[8]
    if nine['games_back_of_ahead_team'] > 4:
        return eight
    elif nine['games_back_of_ahead_team'] <= 4:
        rating1_1, rating2_1, outcome1 = elo_update(eight['elo_value'], nine['elo_value'])
        eight['elo_value'] = rating1_1
        nine['elo_value'] = rating2_1
        if outcome1 == 1:
            return eight
        else:
            rating1_2, rating2_2, outcome2 = elo_update(eight['elo_value'], nine['elo_value'])
            eight['elo_value'] = rating1_2
            nine['elo_value'] = rating2_2
            if outcome2 == 1:
                return eight
            else:
                return nine

In [35]:
# Unit test play in round function
df_western_test = df_western
final_west_team = play_in_round(df_western_test)
print(final_west_team)

index                              17
team_abbrev                       MEM
team_nickname               Grizzlies
elo_basic                        1553
elo_playoff_full                 1456
wins                               35
losses                             38
conference                    western
elo_value                     1511.13
win_percentage               0.479452
standing_min                        8
standing_dense                      7
standing_random                     8
ahead_wins                         42
ahead_losses                       33
games_back_of_ahead_team            6
Name: 7, dtype: object


C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramFiles\Anaconda2

In [36]:
# Assign final playoff spots
west_seed_eight = play_in_round(df_western)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramFiles\Anaconda2

In [37]:
west_seed_eight

index                             10
team_abbrev                      NOP
team_nickname               Pelicans
elo_basic                       1542
elo_playoff_full                1611
wins                              33
losses                            39
conference                   western
elo_value                    1600.42
win_percentage              0.458333
standing_min                       9
standing_dense                     8
standing_random                    9
ahead_wins                        35
ahead_losses                      38
games_back_of_ahead_team         1.5
Name: 8, dtype: object

In [38]:
df_eastern

,index,team_abbrev,team_nickname,elo_basic,elo_playoff_full,wins,losses,conference,elo_value,win_percentage,standing_min,standing_dense,standing_random,ahead_wins,ahead_losses,games_back_of_ahead_team
0,2,MIL,Bucks,1716,1694,58,15,eastern,1693.291963,0.794521,1.0,1.0,1.0,NaN,NaN,NaN
1,6,TOR,Raptors,1679,1628,51,21,eastern,1662.524108,0.708333,2.0,2.0,2.0,58.0,15.0,6.5
2,12,MIA,Heat,1553,1534,47,26,eastern,1594.567978,0.643836,3.0,3.0,3.0,51.0,21.0,4.5
3,5,BOS,Celtics,1628,1682,46,26,eastern,1613.110102,0.638889,4.0,4.0,4.0,47.0,26.0,0.5
4,3,PHI,76ers,1564,1691,45,28,eastern,1650.901135,0.616438,5.0,5.0,5.0,46.0,26.0,1.5
5,13,IND,Pacers,1560,1586,42,31,eastern,1553.228435,0.575342,6.0,6.0,6.0,45.0,28.0,3.0
6,16,BKN,Nets,1478,1560,34,39,eastern,1521.091977,0.465753,7.0,7.0,7.0,42.0,31.0,8.0
7,15,ORL,Magic,1503,1546,33,41,eastern,1503.667754,0.445946,8.0,8.0,8.0,34.0,39.0,1.5
8,20,WAS,Wizards,1396,1432,26,46,eastern,1411.124649,0.361111,9.0,9.0,9.0,33.0,41.0,6.0


In [39]:
east_seed_eight = play_in_round(df_eastern)

In [40]:
#Simulate playoff series
def playoff_series_sim(team_one, team_two):
    team1_wins = 0
    team2_wins = 0
    while team1_wins < 4 and team2_wins < 4:
        rating1, rating2, outcome = elo_update(team_one['elo_value'], team_two['elo_value'])
        team_one['elo_value'] = rating1
        team_two['elo_value'] = rating2
        if outcome == 1:
            team1_wins += 1
        elif outcome == 2:
            team2_wins += 1
    if team1_wins == 4:
        return team_one
    elif team2_wins == 4:
        return team_two
    else:
        return None

In [41]:
winnerofseries = playoff_series_sim(west_seed_eight, east_seed_eight)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [42]:
winnerofseries

index                             15
team_abbrev                      ORL
team_nickname                  Magic
elo_basic                       1503
elo_playoff_full                1546
wins                              33
losses                            41
conference                   eastern
elo_value                    1544.66
win_percentage              0.445946
standing_min                       8
standing_dense                     8
standing_random                    8
ahead_wins                        34
ahead_losses                      39
games_back_of_ahead_team         1.5
Name: 7, dtype: object

In [43]:
#Western playoffs
west18 = playoff_series_sim(west_seed_one, west_seed_eight)
west27 = playoff_series_sim(west_seed_two, west_seed_seven)
west36 = playoff_series_sim(west_seed_three, west_seed_six)
west45 = playoff_series_sim(west_seed_four, west_seed_five)
westsemi1 = playoff_series_sim(west18, west45)
print(westsemi1)
westsemi2 = playoff_series_sim(west27, west36)
print(westsemi2)
westfinals = playoff_series_sim(westsemi1, westsemi2)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


index                              0
team_abbrev                      LAL
team_nickname                 Lakers
elo_basic                       1678
elo_playoff_full                1710
wins                              56
losses                            16
conference                   western
elo_value                     1734.3
win_percentage              0.777778
standing_min                       1
standing_dense                     1
standing_random                    1
ahead_wins                       NaN
ahead_losses                     NaN
games_back_of_ahead_team         NaN
Name: 0, dtype: object
index                              1
team_abbrev                      LAC
team_nickname               Clippers
elo_basic                       1636
elo_playoff_full                1732
wins                              51
losses                            22
conference                   western
elo_value                    1692.63
win_percentage               0.69863
standing_min   

In [44]:
#Eastern playoffs
east18 = playoff_series_sim(east_seed_one, east_seed_eight)
east27 = playoff_series_sim(east_seed_two, east_seed_seven)
east36 = playoff_series_sim(east_seed_three, east_seed_six)
east45 = playoff_series_sim(east_seed_four, east_seed_five)
eastsemi1 = playoff_series_sim(east18, east45)
print(eastsemi1)
eastsemi2 = playoff_series_sim(east27, east36)
print(eastsemi2)
eastfinals = playoff_series_sim(eastsemi1, eastsemi2)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


index                              2
team_abbrev                      MIL
team_nickname                  Bucks
elo_basic                       1716
elo_playoff_full                1694
wins                              58
losses                            15
conference                   eastern
elo_value                    1692.77
win_percentage              0.794521
standing_min                       1
standing_dense                     1
standing_random                    1
ahead_wins                       NaN
ahead_losses                     NaN
games_back_of_ahead_team         NaN
Name: 0, dtype: object
index                              6
team_abbrev                      TOR
team_nickname                Raptors
elo_basic                       1679
elo_playoff_full                1628
wins                              51
losses                            21
conference                   eastern
elo_value                    1685.06
win_percentage              0.708333
standing_min   

In [45]:
print(westfinals)
print(eastfinals)

index                              0
team_abbrev                      LAL
team_nickname                 Lakers
elo_basic                       1678
elo_playoff_full                1710
wins                              56
losses                            16
conference                   western
elo_value                    1766.69
win_percentage              0.777778
standing_min                       1
standing_dense                     1
standing_random                    1
ahead_wins                       NaN
ahead_losses                     NaN
games_back_of_ahead_team         NaN
Name: 0, dtype: object
index                              2
team_abbrev                      MIL
team_nickname                  Bucks
elo_basic                       1716
elo_playoff_full                1694
wins                              58
losses                            15
conference                   eastern
elo_value                    1708.04
win_percentage              0.794521
standing_min   

In [46]:
#NBA Finals
champion = playoff_series_sim(westfinals, eastfinals)

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [47]:
print(champion)

index                              2
team_abbrev                      MIL
team_nickname                  Bucks
elo_basic                       1716
elo_playoff_full                1694
wins                              58
losses                            15
conference                   eastern
elo_value                     1744.3
win_percentage              0.794521
standing_min                       1
standing_dense                     1
standing_random                    1
ahead_wins                       NaN
ahead_losses                     NaN
games_back_of_ahead_team         NaN
Name: 0, dtype: object


In [51]:
#Create counting variable and list to place champions
i = 1
champions_list = []
iterations = 50

while i <= iterations:    
    # Reset season to start
    team_df = team_df_backup
    
    # Do all the season steps
    team_df = all_regular_season(team_df)
    
    # Add win percentage
    team_df['win_percentage'] = team_df['wins']/(team_df['wins']+team_df['losses'])
    
    # Create conference specific dataframes
    df_western = team_df[team_df['conference'] == 'western']
    df_eastern = team_df[team_df['conference'] == 'eastern']
    df_western = bracket_cleaner(df_western)
    df_eastern = bracket_cleaner(df_eastern)
    
    # Create conference seeds
    west_seed_one = df_western.loc[0]
    west_seed_two = df_western.loc[1]
    west_seed_three = df_western.loc[2]
    west_seed_four = df_western.loc[3]
    west_seed_five = df_western.loc[4]
    west_seed_six = df_western.loc[5]
    west_seed_seven = df_western.loc[6]
    west_seed_eight = play_in_round(df_western)
    east_seed_one = df_eastern.loc[0]
    east_seed_two = df_eastern.loc[1]
    east_seed_three = df_eastern.loc[2]
    east_seed_four = df_eastern.loc[3]
    east_seed_five = df_eastern.loc[4]
    east_seed_six = df_eastern.loc[5]
    east_seed_seven = df_eastern.loc[6]
    east_seed_eight = play_in_round(df_eastern)
   
    # Simulate Western playoffs
    west18 = playoff_series_sim(west_seed_one, west_seed_eight)
    west27 = playoff_series_sim(west_seed_two, west_seed_seven)
    west36 = playoff_series_sim(west_seed_three, west_seed_six)
    west45 = playoff_series_sim(west_seed_four, west_seed_five)
    westsemi1 = playoff_series_sim(west18, west45)
    westsemi2 = playoff_series_sim(west27, west36)
    westfinals = playoff_series_sim(westsemi1, westsemi2)

    # Simulate Eastern playoffs
    east18 = playoff_series_sim(east_seed_one, east_seed_eight)
    east27 = playoff_series_sim(east_seed_two, east_seed_seven)
    east36 = playoff_series_sim(east_seed_three, east_seed_six)
    east45 = playoff_series_sim(east_seed_four, east_seed_five)
    eastsemi1 = playoff_series_sim(east18, east45)
    eastsemi2 = playoff_series_sim(east27, east36)
    eastfinals = playoff_series_sim(eastsemi1, eastsemi2)

    champion_winner = playoff_series_sim(westfinals, eastfinals)
    champion = champion_winner['team_abbrev']

    
    # Append champions list
    champions_list.append(champion)
    
    # Increase counter
    i += 1

C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramFiles\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramFiles\Anaconda2\lib\sit

In [52]:
# Get counts of champions
Counter(champions_list)

Counter({'DAL': 1,
         'DEN': 1,
         'LAC': 4,
         'LAL': 20,
         'MIA': 3,
         'MIL': 11,
         'OKC': 5,
         'SAC': 4,
         'TOR': 1})